MIT License

Copyright (c) [year] [fullname]

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

**Import Reference**

In [34]:
import requests
from lxml import html   
import numpy as np
import pandas as pd
import json

**Json Processing**

In [35]:
portfolio = json.load(open("./portfolio.json","r"))
portfolioset = set(portfolio)

**Call FinWiz site**

In [36]:
tickers = ','.join(str(s) for s in portfolioset)
URL = 'https://www.finviz.com/screener.ashx?v=110&t='+tickers
page = requests.get(URL)
tree = html.fromstring(page.content)

In [37]:
screener = tree.xpath('//table[@bgcolor="#d3d3d3"]/tr')

**Post process HTML scrapping**

In [38]:
# screener
header=True
stocks=[]
for stock in screener:
    if header:
        headerarr = []
        headerarr.append("Group")
        headerarr.append(stock.xpath('./td[2]/text()')[0])
        headerarr.append(stock.xpath('./td[3]/text()')[0])
        headerarr.append(stock.xpath('./td[4]/text()')[0])
        headerarr.append(stock.xpath('./td[9]/text()')[0])
        headerarr.append(stock.xpath('./td[10]/text()')[0])
        headerarr.append("Unit Cost")
        headerarr.append("Qty")
        headerarr.append("Total Cost")
        headerarr.append("UnRealized Gain/Loss")
        headerarr.append("UnRealized Gain/Loss %")
        stocks.append(headerarr)
        header=False
    else:
        uc = "0"
        qty = "0"
        unrgl = 0
        unrglpct = 0
        price = stock.xpath('./td[9]/a/span/text()')[0] #Price

        if stock.xpath('./td[2]/a/text()')[0] in portfolioset:
            uc = portfolio[stock.xpath('./td[2]/a/text()')[0]]["unitcost"]
            qty = portfolio[stock.xpath('./td[2]/a/text()')[0]]["qty"]
            unrgl = (float(price) - float(uc)) * float(qty)
            unrglpct = ((float(price) - float(uc)) / float(uc))
            
        arr = [] #stock.xpath('./td/a/text()')
        arr.append(portfolio[stock.xpath('./td[2]/a/text()')[0]]["group"]) #Group
        arr.append(stock.xpath('./td[2]/a/text()')[0]) #Ticker
        arr.append(stock.xpath('./td[3]/a/text()')[0]) #Company
        arr.append(stock.xpath('./td[4]/a/text()')[0]) #Sector
        arr.append("${0}".format(price))
        arr.append(stock.xpath('./td[10]/a/span/text()')[0]) #Change
        arr.append("${0}".format(uc))
        arr.append(qty)
        arr.append("${:.2f}".format(float(uc) * float(qty)))
        arr.append("${:.2f}".format(unrgl))
        arr.append("{:.2%}".format(unrglpct))
        stocks.append(arr)
# print(stocks)

**Attach it to DataFrame**

In [39]:
dataset = pd.DataFrame(stocks)
headers=dataset.iloc[0]
dataset=dataset[1:]
dataset.rename(columns = headers)

,Group,Ticker,Company,Sector,Price,Change,Unit Cost,Qty,Total Cost,UnRealized Gain/Loss,UnRealized Gain/Loss %
1,A,MA,Mastercard Incorporated,Financial,$237.03,-0.77%,$200.00,1,$200.00,$37.03,18.52%
2,A,MSFT,Microsoft Corporation,Technology,$153.83,-0.92%,$120.00,2,$240.00,$67.66,28.19%
